## Optimizing an ML Pipeline in Azure
### Overview
This project is part of the Udacity Azure ML Nanodegree. In this project, we build and optimize an Azure ML pipeline using the Python SDK and a provided Scikit-learn model. This model is then compared to an Azure AutoML run.

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-227131
Azure region: southcentralus
Subscription id: a24a24d5-8d87-4c8a-99b6-91ed2d2df51f
Resource group: aml-quickstarts-227131


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cluster_name = "compute-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    aml_compute = ComputeTarget(workspace=ws, name=cluster_name)
    print('An existing cluster will be used!')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)
    print('An new cluster will be created now!')

aml_compute.wait_for_completion(show_output=True)

An new cluster will be created now!
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': choice(0.1, 0.5, 1, 10, 20, 50),
        '--max_iter': choice(10, 50, 100, 200)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, delay_evaluation = 5, evaluation_interval = 1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                      command=['python', 'train.py'],
                      compute_target=aml_compute,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [4]:
# Submit the hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

RunId: HD_2ba57ffc-2a77-4eb4-8e8c-924c814030f9
Web View: https://ml.azure.com/runs/HD_2ba57ffc-2a77-4eb4-8e8c-924c814030f9?wsid=/subscriptions/a24a24d5-8d87-4c8a-99b6-91ed2d2df51f/resourcegroups/aml-quickstarts-227131/workspaces/quick-starts-ws-227131&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-03-03T23:21:52.213543][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-03-03T23:21:53.5907689Z][SCHEDULER][INFO]Scheduling job, id='HD_2ba57ffc-2a77-4eb4-8e8c-924c814030f9_1' 
[2023-03-03T23:21:53.4406114Z][SCHEDULER][INFO]Scheduling job, id='HD_2ba57ffc-2a77-4eb4-8e8c-924c814030f9_0' 
[2023-03-03T23:21:53.7161920Z][SCHEDULER][INFO]Scheduling job, id='HD_2ba57ffc-2a77-4eb4-8e8c-924c814030f9_2' 
[2023-03-03T23:21:53.764260][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-03-03T23:21:53.8246953Z][SCHEDULER][INFO]Scheduling job, id='HD_2ba57ffc-2a77-4eb4-8e8c-924c814030

In [7]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()


model = best_run.register_model(model_name='Model_HighAccuracy', model_path='./', 
                                properties={'Accuracy': best_run_metrics['Accuracy'],
                                            'Regularization Strength': best_run_metrics['Regularization Strength:'],
                                            'Max Iterations': best_run_metrics['Max iterations:']})

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

dataset = TabularDatasetFactory.from_delimited_files([url])

In [9]:
from train import clean_data

x, y = clean_data(dataset)

In [10]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
automl_config = AutoMLConfig(
    compute_target = aml_compute,
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name='y',
    enable_onnx_compatible_models=True,
    n_cross_validations=2)

In [11]:
# Submit the automl run
automl_run = exp.submit(automl_config, show_output = False)
automl_run.wait_for_completion()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_d8ee9f99-484f-49b3-8700-18a34e923031,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


{'runId': 'AutoML_d8ee9f99-484f-49b3-8700-18a34e923031',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-03-03T23:45:34.899743Z',
 'endTimeUtc': '2023-03-04T00:24:13.183671Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group":"aml-quickstarts-227131","workspace_name":"quick-starts-ws-227131","region":"southcentralus","compute_target":"compute-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_bat

In [12]:
# Retrieve and save your best automl model.
best_run, fitted_model = automl_run.get_output()

best_run_metrics = best_run.get_metrics()

model = best_run.register_model(model_name='AutoML_HighAccuracy', model_path='./')

In [13]:
# Cluster clean up

aml_compute.delete()